In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from scipy.stats import norm, pearsonr
from scipy.integrate import quad
from math import floor, ceil
import pickle
from sklearn.metrics import confusion_matrix
from skimage.metrics import structural_similarity

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle

In [ ]:
from bayesee.evaluation import *
from bayesee.perception import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
subject = "PA"
file_name = repo_path / f"data/search/covert/large-field/p3_data_{subject}.pickle"

with open(file_name, "rb") as f:
    stimulus, response = pickle.load(f)

metadata = stimulus["metadata"]
spot_centers = metadata["spot_centers"]
monitor_width, monitor_height = metadata["monitor_size"]
stimulus_size = metadata["stimulus_size"]
n_location = metadata["n_location"]
spot_size = metadata["spot_size"]
stimulus_ppd = metadata["stimulus_ppd"]
target_amplitude = metadata["target_amplitude"]
target = metadata["target"]

file_name = (
    repo_path / f"data/search/covert/large-field/p2_spatial_statistics_{subject}.csv"
)
spatial_statistics_human = pd.read_csv(file_name)
local_dp = spatial_statistics_human["dp"].values

In [ ]:
human_target = stimulus["df"]["location"].values
human_response = response["df"]["response_location"].values
human_accuracy = human_target == human_response
local_ecc = spatial_statistics_human['ecc'].values
array_index_sort = np.insert(np.argsort(local_ecc)+1,0,0)
human_confusion = confusion_matrix(human_response, human_target, labels=array_index_sort)
human_cr_all = human_confusion[0,0] / sum(human_target==0)
human_hit = np.array([human_confusion[l,l] / sum(human_target == array_index_sort[l]) for l in range(1, n_location)])
human_fa = human_confusion[1:,0] / sum(human_target==0)
human_dp = norm.ppf(human_hit) - norm.ppf(1-human_cr_all)

In [ ]:
n_location_ring = np.array([1,1,6,12])

def confusion_matrix_ring(confusion_mat, n_location_ring):
    # confusion matrix has been sorted by ring
    n_ring = len(n_location_ring) - 1
    cumsum_ring = np.cumsum(n_location_ring)
    
    confusion_ring = np.zeros(4*n_ring+1)
    confusion_diagonal = confusion_mat.diagonal()
    confusion_ring[0] = confusion_diagonal[0]

    for index_ring in range(n_ring):
        array_index_ring = range(cumsum_ring[index_ring], cumsum_ring[index_ring+1])
        hit_ring = confusion_diagonal[array_index_ring].sum()
        miss_ring = confusion_mat[0,array_index_ring].sum()
        fa_ring = confusion_mat[array_index_ring,0].sum()
        fhf_ring = confusion_mat[array_index_ring, 1:].sum() - hit_ring
        confusion_ring[1+index_ring*4:5+index_ring*4] = hit_ring, miss_ring, fa_ring, fhf_ring
    
    return confusion_ring

human_confusion_ring = confusion_matrix_ring(human_confusion, n_location_ring)

In [ ]:
prior = np.array((0.5, *((0.5 / (n_location - 1),) * (n_location - 1))))
assert np.allclose(prior.sum(), 1.0)
log_prior_ratio = np.log(prior / prior[0])
log_likelihood_ratio = np.zeros_like(prior)
model_response = np.zeros_like(human_target)

In [ ]:
n_bootstrap = 1000
array_rmse = np.zeros((n_bootstrap,))
array_mae = np.zeros((n_bootstrap,))
array_ll = np.zeros((n_bootstrap,))
array_pcc = np.zeros((n_bootstrap,))
array_ssim = np.zeros((n_bootstrap,))
array_cs = np.zeros((n_bootstrap,))
array_rmse_ring = np.zeros((n_bootstrap,))
array_mae_ring = np.zeros((n_bootstrap,))
array_ll_ring = np.zeros((n_bootstrap,))
array_pcc_ring = np.zeros((n_bootstrap,))
array_ssim_ring = np.zeros((n_bootstrap,))
array_cs_ring = np.zeros((n_bootstrap,))
array_rmse_diag = np.zeros((n_bootstrap,))
array_mae_diag = np.zeros((n_bootstrap,))
array_ll_diag = np.zeros((n_bootstrap,))
array_pcc_diag = np.zeros((n_bootstrap,))
array_ssim_diag = np.zeros((n_bootstrap,))
array_cs_diag = np.zeros((n_bootstrap,))
array_rmse_dp = np.zeros((n_bootstrap,))
array_mae_dp = np.zeros((n_bootstrap,))
array_pcc_dp = np.zeros((n_bootstrap,))
array_ssim_dp = np.zeros((n_bootstrap,))
array_cs_dp = np.zeros((n_bootstrap,))
array_rmse_hit = np.zeros((n_bootstrap,))
array_mae_hit = np.zeros((n_bootstrap,))
array_pcc_hit = np.zeros((n_bootstrap,))
array_ssim_hit = np.zeros((n_bootstrap,))
array_cs_hit = np.zeros((n_bootstrap,))
array_rmse_fa = np.zeros((n_bootstrap,))
array_mae_fa = np.zeros((n_bootstrap,))
array_pcc_fa = np.zeros((n_bootstrap,))
array_ssim_fa = np.zeros((n_bootstrap,))
array_cs_fa = np.zeros((n_bootstrap,))

In [ ]:
for index_bootstrap in range(n_bootstrap):
    for index_trial in range(len(human_target)):
        array_standard_normal = np.random.normal(size=(n_location - 1,))
        log_likelihood_ratio[1:] = local_dp * (
            array_standard_normal - local_dp / 2
        )

        if human_target[index_trial] > 0:
            log_likelihood_ratio[human_target[index_trial]] +=  local_dp[human_target[index_trial] - 1]**2

        log_posterior_ratio = log_prior_ratio + log_likelihood_ratio
        model_response[index_trial] = np.argmax(log_posterior_ratio)
        
    model_confusion = confusion_matrix(model_response, human_target, labels=array_index_sort)
    model_confusion_probability = (model_confusion + 1) # Laplace smoothing
    model_confusion_probability = model_confusion_probability / model_confusion_probability.sum()
    
    model_confusion_ring = confusion_matrix_ring(model_confusion, n_location_ring)
    model_confusion_ring_probability = (model_confusion_ring + 1) # Laplace smoothing
    model_confusion_ring_probability = model_confusion_ring_probability / model_confusion_ring_probability.sum()

    model_cr_all = model_confusion[0,0] / sum(human_target==0)
    model_hit = np.array([model_confusion[l,l] / sum(human_target == array_index_sort[l]) for l in range(1, n_location)])
    model_fa = model_confusion[1:,0] / sum(human_target==0)
    model_dp = norm.ppf(model_hit) - norm.ppf(1-model_cr_all)

    array_rmse[index_bootstrap] = rmse(human_confusion, model_confusion)
    array_mae[index_bootstrap] = mae(human_confusion, model_confusion)
    array_ll[index_bootstrap] = (human_confusion * np.log(model_confusion_probability)).sum()
    array_pcc[index_bootstrap] = pearson(human_confusion, model_confusion)
    array_ssim[index_bootstrap] = ssim(human_confusion, model_confusion)
    array_cs[index_bootstrap] = cosine_similarity(human_confusion, model_confusion)
    
    array_rmse_ring[index_bootstrap] = rmse(human_confusion_ring, model_confusion_ring)
    array_mae_ring[index_bootstrap] = mae(human_confusion_ring, model_confusion_ring)
    array_ll_ring[index_bootstrap] = (human_confusion_ring * np.log(model_confusion_ring_probability)).sum()
    array_pcc_ring[index_bootstrap] = pearson(human_confusion_ring, model_confusion_ring)
    array_ssim_ring[index_bootstrap] = ssim(human_confusion_ring, model_confusion_ring)
    array_cs_ring[index_bootstrap] = cosine_similarity(human_confusion_ring, model_confusion_ring)
    
    array_rmse_diag[index_bootstrap] = rmse(human_confusion.diagonal(), model_confusion.diagonal())
    array_mae_diag[index_bootstrap] = mae(human_confusion.diagonal(), model_confusion.diagonal())
    array_ll_diag[index_bootstrap] = (human_confusion.diagonal() * np.log(model_confusion_probability.diagonal())).sum()
    array_pcc_diag[index_bootstrap] = pearson(human_confusion.diagonal(), model_confusion.diagonal())
    array_ssim_diag[index_bootstrap] = ssim(human_confusion.diagonal(), model_confusion.diagonal())
    array_cs_diag[index_bootstrap] = cosine_similarity(human_confusion.diagonal(), model_confusion.diagonal())
    
    array_rmse_dp[index_bootstrap] = rmse(human_dp, model_dp)
    array_mae_dp[index_bootstrap] = mae(human_dp, model_dp)
    array_pcc_dp[index_bootstrap] = pearson(human_dp, model_dp)
    array_ssim_dp[index_bootstrap] = ssim(human_dp, model_dp)
    array_cs_dp[index_bootstrap] = cosine_similarity(human_dp, model_dp)
    
    array_rmse_hit[index_bootstrap] = rmse(human_hit, model_hit)
    array_mae_hit[index_bootstrap] = mae(human_hit, model_hit)
    array_pcc_hit[index_bootstrap] = pearson(human_hit, model_hit)
    array_ssim_hit[index_bootstrap] = ssim(human_hit, model_hit)
    array_cs_hit[index_bootstrap] = cosine_similarity(human_hit, model_hit)
    
    array_rmse_fa[index_bootstrap] = rmse(human_fa, model_fa)
    array_mae_fa[index_bootstrap] = mae(human_fa, model_fa)
    array_pcc_fa[index_bootstrap] = pearson(human_fa, model_fa)
    array_ssim_fa[index_bootstrap] = ssim(human_fa, model_fa)
    array_cs_fa[index_bootstrap] = cosine_similarity(human_fa, model_fa)

In [ ]:
print(
    f"""
| Cost | Mean | SD | 2.5th | 97.5th |
| :---: | :---: | :---: | :---: | :---: |
| Root Mean Squared Error (RMSE) | {array_rmse.mean():.3f} | {array_rmse.std():.3f} | {np.percentile(array_rmse, 2.5):.3f} | {np.percentile(array_rmse, 97.5):.3f} |
| Mean Absolute Error (MAE) | {array_mae.mean():.3f} | {array_mae.std():.3f} | {np.percentile(array_mae, 2.5):.3f} | {np.percentile(array_mae, 97.5):.3f} |
| Log Likelihood (LL) | {array_ll.mean():.3f} | {array_ll.std():.3f} | {np.percentile(array_ll, 2.5):.3f} | {np.percentile(array_ll, 97.5):.3f} |
| Pearson Correlation Coefficient (PCC) | {array_pcc.mean():.3f} | {array_pcc.std():.3f} | {np.percentile(array_pcc, 2.5):.3f} | {np.percentile(array_pcc, 97.5):.3f} |
| Structural Similarity Index Measure (SSIM) | {array_ssim.mean():.3f} | {array_ssim.std():.3f} | {np.percentile(array_ssim, 2.5):.3f} | {np.percentile(array_ssim, 97.5):.3f} |
| Cosine Similarity (CS) | {array_cs.mean():.3f} | {array_cs.std():.3f} | {np.percentile(array_cs, 2.5):.3f} | {np.percentile(array_cs, 97.5):.3f} |
| Ring Root Mean Squared Error (RMSE) | {array_rmse_ring.mean():.3f} | {array_rmse_ring.std():.3f} | {np.percentile(array_rmse_ring, 2.5):.3f} | {np.percentile(array_rmse_ring, 97.5):.3f} |
| Ring Mean Absolute Error (MAE) | {array_mae_ring.mean():.3f} | {array_mae_ring.std():.3f} | {np.percentile(array_mae_ring, 2.5):.3f} | {np.percentile(array_mae_ring, 97.5):.3f} |
| Ring Log Likelihood (LL) | {array_ll_ring.mean():.3f} | {array_ll_ring.std():.3f} | {np.percentile(array_ll_ring, 2.5):.3f} | {np.percentile(array_ll_ring, 97.5):.3f} |
| Ring Pearson Correlation Coefficient (PCC) | {array_pcc_ring.mean():.3f} | {array_pcc_ring.std():.3f} | {np.percentile(array_pcc_ring, 2.5):.3f} | {np.percentile(array_pcc_ring, 97.5):.3f} |
| Ring Structural Similarity Index Measure (SSIM) | {array_ssim_ring.mean():.3f} | {array_ssim_ring.std():.3f} | {np.percentile(array_ssim_ring, 2.5):.3f} | {np.percentile(array_ssim_ring, 97.5):.3f} |
| Ring Cosine Similarity (CS) | {array_cs_ring.mean():.3f} | {array_cs_ring.std():.3f} | {np.percentile(array_cs_ring, 2.5):.3f} | {np.percentile(array_cs_ring, 97.5):.3f} |
| Diagonal Root Mean Squared Error (RMSE) | {array_rmse_diag.mean():.3f} | {array_rmse_diag.std():.3f} | {np.percentile(array_rmse_diag, 2.5):.3f} | {np.percentile(array_rmse_diag, 97.5):.3f} |
| Diagonal Mean Absolute Error (MAE) | {array_mae_diag.mean():.3f} | {array_mae_diag.std():.3f} | {np.percentile(array_mae_diag, 2.5):.3f} | {np.percentile(array_mae_diag, 97.5):.3f} |
| Diagonal Log Likelihood (LL) | {array_ll_diag.mean():.3f} | {array_ll_diag.std():.3f} | {np.percentile(array_ll_diag, 2.5):.3f} | {np.percentile(array_ll_diag, 97.5):.3f} |
| Diagonal Pearson Correlation Coefficient (PCC) | {array_pcc_diag.mean():.3f} | {array_pcc_diag.std():.3f} | {np.percentile(array_pcc_diag, 2.5):.3f} | {np.percentile(array_pcc_diag, 97.5):.3f} |
| Diagonal Structural Similarity Index Measure (SSIM) | {array_ssim_diag.mean():.3f} | {array_ssim_diag.std():.3f} | {np.percentile(array_ssim_diag, 2.5):.3f} | {np.percentile(array_ssim_diag, 97.5):.3f} |
| Diagonal Cosine Similarity (CS) | {array_cs_diag.mean():.3f} | {array_cs_diag.std():.3f} | {np.percentile(array_cs_diag, 2.5):.3f} | {np.percentile(array_cs_diag, 97.5):.3f} |
| D' Root Mean Squared Error (RMSE) | {array_rmse_dp.mean():.3f} | {array_rmse_dp.std():.3f} | {np.percentile(array_rmse_dp, 2.5):.3f} | {np.percentile(array_rmse_dp, 97.5):.3f} |
| D' Mean Absolute Error (MAE) | {array_mae_dp.mean():.3f} | {array_mae_dp.std():.3f} | {np.percentile(array_mae_dp, 2.5):.3f} | {np.percentile(array_mae_dp, 97.5):.3f} |
| D' Pearson Correlation Coefficient (PCC) | {array_pcc_dp.mean():.3f} | {array_pcc_dp.std():.3f} | {np.percentile(array_pcc_dp, 2.5):.3f} | {np.percentile(array_pcc_dp, 97.5):.3f} |
| D' Structural Similarity Index Measure (SSIM) | {array_ssim_dp.mean():.3f} | {array_ssim_dp.std():.3f} | {np.percentile(array_ssim_dp, 2.5):.3f} | {np.percentile(array_ssim_dp, 97.5):.3f} |
| D' Cosine Similarity (CS) | {array_cs_dp.mean():.3f} | {array_cs_dp.std():.3f} | {np.percentile(array_cs_dp, 2.5):.3f} | {np.percentile(array_cs_dp, 97.5):.3f} |
| Hit Root Mean Squared Error (RMSE) | {array_rmse_hit.mean():.3f} | {array_rmse_hit.std():.3f} | {np.percentile(array_rmse_hit, 2.5):.3f} | {np.percentile(array_rmse_hit, 97.5):.3f} |
| Hit Mean Absolute Error (MAE) | {array_mae_hit.mean():.3f} | {array_mae_hit.std():.3f} | {np.percentile(array_mae_hit, 2.5):.3f} | {np.percentile(array_mae_hit, 97.5):.3f} |
| Hit Pearson Correlation Coefficient (PCC) | {array_pcc_hit.mean():.3f} | {array_pcc_hit.std():.3f} | {np.percentile(array_pcc_hit, 2.5):.3f} | {np.percentile(array_pcc_hit, 97.5):.3f} |
| Hit Structural Similarity Index Measure (SSIM) | {array_ssim_hit.mean():.3f} | {array_ssim_hit.std():.3f} | {np.percentile(array_ssim_hit, 2.5):.3f} | {np.percentile(array_ssim_hit, 97.5):.3f} |
| Hit Cosine Similarity (CS) | {array_cs_hit.mean():.3f} | {array_cs_hit.std():.3f} | {np.percentile(array_cs_hit, 2.5):.3f} | {np.percentile(array_cs_hit, 97.5):.3f} |
| False Alarm Root Mean Squared Error (RMSE) | {array_rmse_fa.mean():.3f} | {array_rmse_fa.std():.3f} | {np.percentile(array_rmse_fa, 2.5):.3f} | {np.percentile(array_rmse_fa, 97.5):.3f} |
| False Alarm Mean Absolute Error (MAE) | {array_mae_fa.mean():.3f} | {array_mae_fa.std():.3f} | {np.percentile(array_mae_fa, 2.5):.3f} | {np.percentile(array_mae_fa, 97.5):.3f} |
| False Alarm Pearson Correlation Coefficient (PCC) | {array_pcc_fa.mean():.3f} | {array_pcc_fa.std():.3f} | {np.percentile(array_pcc_fa, 2.5):.3f} | {np.percentile(array_pcc_fa, 97.5):.3f} |
| False Alarm Structural Similarity Index Measure (SSIM) | {array_ssim_fa.mean():.3f} | {array_ssim_fa.std():.3f} | {np.percentile(array_ssim_fa, 2.5):.3f} | {np.percentile(array_ssim_fa, 97.5):.3f} |
| False Alarm Cosine Similarity (CS) | {array_cs_fa.mean():.3f} | {array_cs_fa.std():.3f} | {np.percentile(array_cs_fa, 2.5):.3f} | {np.percentile(array_cs_fa, 97.5):.3f} |

(Bootstrapped {n_bootstrap} times.)
"""
)